In [ ]:
!pip install -q \
    "langchain>=1.0.2,<2.0.0" \
    langchain-openai \
    langchain-tavily \
    numpy scikit-learn scipy \
    "FlagEmbedding>=1.3.0"


In [ ]:
import importlib
import langchain_core.tools
importlib.reload(langchain_core.tools)  # перезагружаем на случай кэша
from langchain_core.tools import tool as langchain_tool
import os, json, re, warnings, numpy as np
from datetime import datetime
from typing import List, Dict, Any, Optional
from langchain_core.messages import HumanMessage, AIMessage
from langchain_openai import ChatOpenAI
from langchain_tavily import TavilySearch
from FlagEmbedding import BGEM3FlagModel

warnings.filterwarnings("ignore", category=UserWarning, module="langchain")

# 🔑 Загружаем и чистим ключи (удаляем кавычки/пробелы)
if not os.path.exists('key.env'):
    raise FileNotFoundError("❌ Загрузите key.env через 📁 → Upload")
with open('key.env', encoding='utf-8') as f:
    for line in f:
        line = line.strip()
        if line and '=' in line and not line.startswith('#'):
            k, v = line.split('=', 1)
            os.environ[k.strip()] = v.strip().strip('"').strip("'")  # ✅ чистим кавычки

openai_key = os.getenv("OPENAI_API_KEY")
tavily_key = os.getenv("TAVILY_API_KEY")
if not (openai_key and tavily_key):
    raise ValueError("❌ OPENAI_API_KEY и TAVILY_API_KEY не найдены в key.env")

# 🧠 Инициализация
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0.3, max_tokens=512)
search_tool = TavilySearch(max_results=3)

# ----------------------------
# 🛠️ Инструменты
# ----------------------------
@langchain_tool
def hybrid_search_tool(query: str, top_k: int = 3) -> List[str]:
    """Ищет подходящее программное обеспечение по запросу (Python, R и др.) с помощью BGE-M3 hybrid search."""
    if not hasattr(hybrid_search_tool, "_emb_model"):
        print("🧠 Загружаю BAAI/bge-m3 через FlagEmbedding…")
        hybrid_search_tool._emb_model = BGEM3FlagModel('BAAI/bge-m3', use_fp16=False, device='cpu')
        hybrid_search_tool._allowed = ["Python", "R", "Julia", "MATLAB", "SPSS", "SAS", "Stata", "Wolfram Mathematica", "JASP", "PSPP"]
        embs = hybrid_search_tool._emb_model.encode(
            hybrid_search_tool._allowed,
            return_dense=True,
            return_sparse=True,
            return_colbert_vecs=False
        )
        hybrid_search_tool._dense = np.array(embs["dense_vecs"])
        hybrid_search_tool._sparse = embs.get("sparse_vecs")
        hybrid_search_tool._use_hybrid = hybrid_search_tool._sparse is not None
        from sklearn.preprocessing import normalize
        hybrid_search_tool._dense_norm = normalize(hybrid_search_tool._dense, norm='l2')

    def _norm(x):
        if x.max() == x.min(): return np.zeros_like(x)
        return (x - x.min()) / (x.max() - x.min() + 1e-8)

    m = hybrid_search_tool._emb_model
    allowed = hybrid_search_tool._allowed
    dense_db = hybrid_search_tool._dense_norm
    sparse_db = hybrid_search_tool._sparse
    use_hybrid = hybrid_search_tool._use_hybrid

    q_emb = m.encode([query], return_dense=True, return_sparse=True, return_colbert_vecs=False)
    q_dense = normalize(np.array(q_emb["dense_vecs"]), norm='l2')[0]
    dense_scores = dense_db @ q_dense

    if use_hybrid and sparse_db is not None:
        q_sparse = q_emb.get("sparse_vecs", [None])[0]
        if q_sparse is not None:
            sparse_scores = sparse_db.dot(q_sparse.T).toarray().flatten()
            dense_scores = _norm(dense_scores)
            sparse_scores = _norm(sparse_scores)
            scores = 0.7 * dense_scores + 0.3 * sparse_scores
        else:
            scores = dense_scores
    else:
        scores = dense_scores

    top_idx = np.argsort(scores)[::-1][:top_k]
    return [allowed[i] for i in top_idx]


@langchain_tool
def web_search_tool(query: str) -> List[Dict[str, str]]:
    """Ищет информацию в интернете через Tavily (макс. 3 результата)."""
    raw = search_tool.invoke(query)
    results = raw.get("results", raw) if isinstance(raw, dict) else raw
    if not isinstance(results, list):
        return [{"title": "Ошибка", "url": "#", "content": "Нет данных"}]
    return [
        {
            "title": r.get("title", "").strip(),
            "url": r.get("url", "").strip(),
            "content": r.get("content", "")[:250].strip()
        }
        for r in results[:3]
    ]


@langchain_tool
def generate_report_tool(topic: str, programs: List[Dict[str, str]]) -> Dict[str, str]:
    """Генерирует и сохраняет HTML+MD отчёт по теме и списку программ. LLM НЕ генерирует HTML."""
    os.makedirs("reports", exist_ok=True)
    ts = datetime.now().strftime("%Y%m%d_%H%M%S")
    base = f"report_{topic.lower().replace(' ', '_').replace('+', 'and')}_{ts}"

    # Генерация контекста на лету (без участия LLM)
    context = f"Тема отчёта: **{topic}**\n\nНайдено {len(programs)} программ:\n"
    for p in programs:
        context += f"- **{p.get('name', p.get('title', 'N/A'))}**\n  → {p.get('url', '#')}\n  {p.get('description', p.get('content', ''))[:200]}\n\n"

    # HTML
    html_items = []
    for p in programs[:5]:
        name = p.get("name", p.get("title", "N/A"))
        url = p.get("url", "#")
        desc = p.get("description", p.get("content", ""))[:200].replace("&", "&amp;").replace("<", "&lt;").replace(">", "&gt;")
        html_items.append(f"<div><strong><a href='{url}' target='_blank'>{name}</a></strong><br>{desc}…</div>")

    html = f"""<!DOCTYPE html><html lang="ru"><head><meta charset="utf-8"><title>{topic}</title>
<style>body{{font-family:Arial,sans-serif;line-height:1.6;max-width:800px;margin:40px auto}}
.report{{background:#f9f9f9;padding:20px;border-radius:5px;white-space:pre-wrap}}</style></head>
<body><h1>📊 Отчёт: <code>{topic}</code></h1><p><i>{datetime.now().strftime('%d.%m.%Y %H:%M')}</i></p>
<h2>📋 Программы</h2>{''.join(html_items)}</body></html>"""

    html_path = f"reports/{base}.html"
    with open(html_path, "w", encoding="utf-8") as f:
        f.write(html)

    # MD
    md_items = []
    for p in programs[:5]:
        name = p.get("name", p.get("title", "N/A"))
        url = p.get("url", "#")
        desc = p.get("description", p.get("content", ""))[:120].replace("\n", " ").strip()
        md_items.append(f"- **[{name}]({url})** — {desc}…")

    md = f"# Отчёт: `{topic}`\n\n> {datetime.now().strftime('%d.%m.%Y %H:%M')}\n\n## Программы\n" + "\n".join(md_items)
    md_path = f"reports/{base}.md"
    with open(md_path, "w", encoding="utf-8") as f:
        f.write(md)

    print(f"✅ Отчёты:\n   📄 {html_path}\n   📝 {md_path}")
    return {"html_path": html_path, "md_path": md_path}


TOOLS_DICT = {
    "hybrid_search": hybrid_search_tool,
    "web_search": web_search_tool,
    "generate_report": generate_report_tool,
}

# ----------------------------
# 🧰 MCP-обёртки (теперь через .invoke())
# ----------------------------
def mcp_hybrid_search(args: Dict[str, Any]) -> Dict[str, Any]:
    try:
        result = hybrid_search_tool.invoke(args)
        return {"status": "success", "result": result}
    except Exception as e:
        return {"status": "error", "message": str(e)}

def mcp_web_search(args: Dict[str, Any]) -> Dict[str, Any]:
    try:
        result = web_search_tool.invoke(args)
        return {"status": "success", "result": result}
    except Exception as e:
        return {"status": "error", "message": str(e)}

def mcp_generate_report(args: Dict[str, Any]) -> Dict[str, Any]:
    try:
        result = generate_report_tool.invoke(args)
        return {"status": "success", "result": result}
    except Exception as e:
        return {"status": "error", "message": str(e)}

MCP_TOOLS = {
    "hybrid_search": mcp_hybrid_search,
    "web_search": mcp_web_search,
    "generate_report": mcp_generate_report,
}

# ----------------------------
# 🔁 ReAct с безопасным JSON-парсингом
# ----------------------------
def safe_parse_json(text: str) -> Optional[dict]:
    # Ищем первый { и последний }
    start = text.find("{")
    end = text.rfind("}") + 1
    if start == -1 or end == 0:
        return None
    try:
        return json.loads(text[start:end])
    except Exception as e:
        # Попытка исправить обрезанные URL и кавычки
        clean = text[start:end]
        clean = re.sub(r'(?<!\\)"(\s*:)', r'"\1', clean)  # избегаем \": → "
        clean = re.sub(r'\\n', ' ', clean)
        try:
            return json.loads(clean)
        except:
            return None

def run_react(query: str, mode: str = "langchain"):
    tools_desc = """- hybrid_search(query: str, top_k: int = 3) → список названий программ
- web_search(query: str) → список источников: [{"title", "url", "content"}]
- generate_report(topic: str, programs: list) → {"html_path", "md_path"}"""

    system_prompt = f"""Ты — ReAct-агент для поиска и составления отчётов о **компьютерных программах и языках программирования**
    (например: Python, R, Julia, MATLAB), а НЕ об образовательных курсах.
❗Правила:
У тебя есть ТОЛЬКО ЭТИ инструменты:
{tools_desc}
❗ВАЖНО:
- Вызывай ТОЛЬКО инструменты из списка выше.
- Используй ТОЧНЫЕ имена: "hybrid_search", "web_search", "generate_report".
- НЕ выдумывай новые имена (например, "search_courses", "find_programs" — ЗАПРЕЩЕНО).
- Если не уверен — вызови "hybrid_search" с уточняющим запросом.

1. Думай → Действуй → Наблюдай.
2. Вызывай ТОЛЬКО один инструмент за раз.
3. Формат действия:
Action: {{"tool": "имя", "arguments": {{"ключ": "значение"}}}}
4. В arguments — ТОЛЬКО простые типы (str, int, list/dict без HTML!).
5. Для generate_report передавай: {{"topic": "...", "programs": [{{"name":"...", "url":"...", "description":"..."}}]}}

Цель: {query}
"""

    messages = [
        ("system", system_prompt),
        ("human", query),
    ]

    for step in range(1, 9):
        response = llm.invoke(messages)
        text = response.content.strip()
        print(f"\n🔍 Step {step} | LLM:\n{text}")

        if "Final Answer:" in text:
            final = text.split("Final Answer:", 1)[1].strip()
            print(f"\n🏆 Final Answer:\n{final}")
            return final

        # Парсим Action
        if "Action:" not in text:
            print("⚠️ Нет 'Action:' — завершаю.")
            break

        action_part = text.split("Action:", 1)[1].strip()
        action_json = safe_parse_json(action_part)
        if not action_json:
            print("❌ Невозможно распарсить JSON. Пробую извлечь инструмент по имени.")
            tool_name = "hybrid_search"
            args = {"query": query, "top_k": 3}
        else:
            tool_name = action_json.get("tool", "")
            args = action_json.get("arguments", {})

        # Вызов инструмента
        if mode == "langchain":
          tool_func = TOOLS_DICT.get(tool_name)
          if tool_func:
            try:
              result = tool_func.invoke(args)
              obs = {"status": "success", "result": result}
            except Exception as e:
              obs = {"status": "error", "message": str(e)}
          else:
            obs = {"status": "error", "message": f"Инструмент '{tool_name}' не найден."}
        else:  # mcp
            handler = MCP_TOOLS.get(tool_name)
            if handler:
                try:
                    obs = handler(args)
                except Exception as e:
                    obs = {"status": "error", "message": str(e)}
            else:
                obs = {"status": "error", "message": f"MCP инструмент '{tool_name}' не найден."}

        observation = json.dumps(obs, ensure_ascii=False, indent=2)
        print(f"\n👁️ Observation:\n{observation}")

        messages.append(AIMessage(content=text))
        messages.append(("human", f"Observation: {observation}"))

    return "❌ Превышено число шагов."

# ▶️ ЗАПУСК
if __name__ == "__main__":
    query = input("💬 О чём отчёт? > ").strip()
    mode = input("🛠️ Режим: [langchain] / mcp > ").strip().lower()
    if mode not in ("langchain", "mcp"):
        mode = "langchain"
    print("\n🔍 Проверка имён инструментов:")
    for t in [hybrid_search_tool, web_search_tool, generate_report_tool]:
      print(f"- {t.__class__.__name__}: .name = '{t.name}' | .description = {repr(t.description[:60])}...")
    print(f"\n🚀 Запуск ReAct ({mode}) для: '{query}'\n")
    run_react(query, mode=mode)

💬 О чём отчёт? > р
🛠️ Режим: [langchain] / mcp > mcp

🔍 Проверка имён инструментов:
- StructuredTool: .name = 'hybrid_search_tool' | .description = 'Ищет подходящее программное обеспечение по запросу (Python, '...
- StructuredTool: .name = 'web_search_tool' | .description = 'Ищет информацию в интернете через Tavily (макс. 3 результата'...
- StructuredTool: .name = 'generate_report_tool' | .description = 'Генерирует и сохраняет HTML+MD отчёт по теме и списку програ'...

🚀 Запуск ReAct (mcp) для: 'р'


🔍 Step 1 | LLM:
Action: {"tool": "hybrid_search", "arguments": {"query": "computer programs and programming languages"}}
🧠 Загружаю BAAI/bge-m3 через FlagEmbedding…


Fetching 30 files:   0%|          | 0/30 [00:00<?, ?it/s]

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.



👁️ Observation:
{
  "status": "success",
  "result": [
    "Python",
    "Wolfram Mathematica",
    "MATLAB"
  ]
}

🔍 Step 2 | LLM:
Action: {"tool": "web_search", "arguments": {"query": "Python programming language overview"}}

👁️ Observation:
{
  "status": "success",
  "result": [
    {
      "title": "Python - Overview - Tutorials Point",
      "url": "https://www.tutorialspoint.com/python/python_overview.htm",
      "content": "# Python - Overview. Python is a high-level, interpreted, interactive and object-oriented scripting language. Python is designed to be highly readable. * **Python is Interactive** − You can actually sit at a Python prompt and interact with the interp"
    },
    {
      "title": "What is Python? Executive Summary",
      "url": "https://www.python.org/doc/essays/blurb/",
      "content": "**Notice:** While JavaScript is not essential for this website, your interaction with the content will be limited. Please turn JavaScript on for the full experience. # What